In [1]:
#Step1　把遊記從資料庫倒出來


#####################################################################
#一篇遊記提及兩個以上的景點   因為cursor為指向概念 所以這邊要設定兩個sql cursor
def squary(IP,User,PWd,DB,sql,folder):
    import MySQLdb
    import os
    import re
    os.popen('mkdir '+folder)
    db = MySQLdb.connect(IP,User,PWd,DB,charset='utf8')#連結資料庫
    count=0
    CntDic = {}
   
    cursor = db.cursor()
    TmpSet = set()

    fid2 =folder
    Set = set()
    rowdic={}    #先建立第一個字典存放所有景點的key與value(被提及次數)
    try:
        cursor.execute(sql)  #執行sql指令
        results = cursor.fetchall() #看結果
        for row in results:
            if not str(row[0]) in rowdic:  #因為key值不能為value 所以要先轉成str
                rowdic[str(row[0])]=1
            else:
                rowdic[str(row[0])]=rowdic[str(row[0])]+1
    except KeyError:
        print "Error: unable to fecth data"                
#    return rowdic
    db = MySQLdb.connect(IP,User,PWd,DB,charset='utf8')#連結資料庫  第二段sql指令
    count=0
    cursor1 = db.cursor()
    rrrdic={}  #第二個字典為存放另一個Key與value(被提及數)
    try:
        cursor1.execute(sql)
        results1 = cursor1.fetchall()
        for row in results1:
            count+=1
            mount=rowdic.get(str(row[0]))  #先取得第一個字典的value值
#            print mount
            if not str(row[0]) in rrrdic:   #若不在第二個字典裡則有初始值1
                rrrdic[str(row[0])]=1
                if rrrdic.get(str(row[0])) != mount:
                    with open(fid2+'/'+str(row[0])+'.txt','w') as f:
                        f.write(row[3].encode('utf-8')+' ')
                else:    
                    with open(fid2+'/'+str(row[0])+'.txt','a') as f2:
                        f2.write(row[3].encode('utf-8')+'\n'+row[1].encode('utf-8'))
    #                    Set.add(row[0])
            else:   #若在第二個字典裡 則數量加1
                rrrdic[str(row[0])]=rrrdic[str(row[0])]+1
                if rrrdic.get(str(row[0]))!=mount:  #判別value在第一與第二個字典裡是否相同 若不相同則指寫入景點
                    with open(fid2+'/'+str(row[0])+'.txt','a') as f3:
                        f3.write(row[3].encode('utf-8')+' ')
                        count += 1
                else: #若相同則把最後一個景點與遊記寫入
                    with open(fid2+'/'+str(row[0])+'.txt','a') as f4:
                        f4.write(row[3].encode('utf-8')+'\n'+row[1].encode('utf-8'))
                        #print row[1]                             
    except KeyError:
        print "Error: unable to fecth data"

    #print rrrdic
    #print "查詢數量:"+str(count)
    z1 = 0
    for z in os.listdir(folder):
        z1 += 1
    print 'Articles Be Downloaded:',z1
#####################################################################


#Step2　把評論.遊記從資料庫倒出來


#####################################################################
def art(IP,User,PWd,DB,sqlart,fid2):#遊記
    import os
    import MySQLdb
    os.popen('mkdir '+fid2)
    db1 = MySQLdb.connect(IP,User,PWd,DB,charset='utf8')#連結資料庫  第二段sql指令
    count=0

    #找出提及一個景點以上的遊記

    fid2 = fid2 + '/'
    cursor1 = db1.cursor()
    attdic={} 
    try:

        cursor1.execute(sqlart)
        results1 = cursor1.fetchall()

        for row in results1:
            if not str(row[2]) in attdic:   #若此景點不再字典初始值為0
                attdic[str(row[2])]=1
                with open(fid2+str(row[2])+'.txt','w') as f:  #以景點編號取檔名寫入檔案中
                    f.write(row[3].encode('utf8').strip()+','+row[1].encode('utf-8').strip()+'\n')
    #                    Set.add(row[0])

            else:  
                attdic[str(row[2])]=attdic[str(row[2])]+1 #若字典有則加1
                with open(fid2+str(row[2])+'.txt','a') as f2:  #後面增加其他的遊記
                    f2.write(row[3].encode('utf8').strip()+','+row[1].encode('utf-8').strip()+'\n')
                    count += 1                           

    except KeyError:
        print "Error: unable to fecth data"
    
    print 'Total Reading Articles(Not Distinct):',sum(attdic.values())
    
def com(IP,User,PWd,DB,sqlcom,fid3):#評論 
    import MySQLdb
    db = MySQLdb.connect(IP,User,PWd,DB,charset='utf8')#連結資料庫  第二段sql指令
    count=0

    #找出提及一個景點以上的評論
    
    fid3 = fid3 + '/'
    cursor1 = db.cursor()
    comdic={} 
    try:

        cursor1.execute(sqlcom)
        results1 = cursor1.fetchall()
        for row in results1:
            if not str(row[0]) in comdic:   #若此景點不再字典初始值為0
                comdic[str(row[0])]=1
                with open(fid3+str(row[0])+'.txt','a') as f:  #以景點編號取檔名寫入檔案中
                    f.write(row[1].encode('utf8').strip()+','+row[2].encode('utf-8').strip()+'\n')
    #                    Set.add(row[0])

            else:  
                comdic[str(row[0])]=comdic[str(row[0])]+1 #若字典有則加1
                with open(fid3+str(row[0])+'.txt','a') as f2:  #後面增加其他的遊記
                    f2.write(row[1].encode('utf8').strip()+','+row[2].encode('utf-8').strip()+'\n')
                    count += 1                           

    except KeyError:
        print "Error: unable to fecth data"
        
    print 'Total Reading Comments:',sum(comdic.values())
#####################################################################


#Step3　把評論.遊記全部結合一起


#####################################################################
def CombineAnC(AnCfid,InputFile):
    for a in os.listdir(AnCfid):
        with open(InputFile,'a') as ft:
            with open(AnCfid+'/'+a,'r') as f:
                ft.write(f.read()+'\n')
#####################################################################


#Step4　訓練W2V模型


#####################################################################
def TrainW2V(InputFile,OutputFile):
    import re
    with open(OutputFile,'w') as f:
        cnt = 0
        for i in open(InputFile,'r'):
            
        ### 舊程式碼 #########################################################
        #    if re.match('(.*?)(,)(.*)',i):
        #        x = re.match('(.*?)(,)(.*)',i)
        #        words = jieba.cut(x.group(3))
        #        f.write(x.group(1)+' '+' '.join(words).encode('utf-8')+'\n')
        #        cnt += 1       
        #####################################################################
        
        ### 新程式碼 #########################################################
            if re.match('(.*?)(,)(.*)',i):#　有逗號的才會進來（有去除空行的作用）
                x = re.match('(.*?)(,)(.*)',i)
                words = jieba.cut(x.group(3))

                if re.match('(.*?)(\()(.*?)(\))',x.group(1)):
                    y = re.match('(.*?)(\()(.*?)(\))',x.group(1))
                    z = y.group(1)+' '+y.group(2)+' '+y.group(3)+' '+y.group(4)

                    f.write(z+' '+' '.join(words).encode('utf-8')+'\n')
                else:
                    f.write(x.group(1)+' '+' '.join(words).encode('utf-8')+'\n')
                    
                cnt += 1
        #####################################################################
        print 'Read Lines:',cnt
        print 'InputFile Len:',len([line for line in open(InputFile)])
        print 'OutputFile Len:',len([line for line in open(OutputFile)])

    ####################################################################
    #訓練model
    import os
    import gensim
    import subprocess
    command = 'python train_word2vec_model.py {} {}.model {}.vector'
    subprocess.check_call(command.format(OutputFile,\
                         OutputFile.split('.txt')[0],\
                         OutputFile.split('.txt')[0]))
    time.sleep(5)
#####################################################################


#Step5　利用W2V模型對景點做關鍵字淬取


#####################################################################
def artsco(W2V_Art_Model,Folder,Rank,exdic):
    import os
    import operator
    import gensim
    model = gensim.models.Word2Vec.load(W2V_Art_Model)
    
    filePath=[]#　存放遊記的路徑
    cnt=0#　計算總關鍵字量
    
    for name in os.listdir(Folder):
        filePath.append(Folder+'/'+name)

    Dica = {}#　先開一個大字典 裡放要放多個小字典
    for k in filePath:
        Dic={}#　每一個檔案兩個景點開一個字典
        line=open(k,'r').readline().strip()
        sp=line.split(' ')
        for z in range(0,len(sp)):
            Dic[sp[z].strip()] = ''

    ###　從外部調用JiebaTFIDF的方法，並將結果(陣列)存起來
        jie=jiebaNoEng(k,Rank)
        
        for i in Dic:#　Dic內容(i)的編碼是UTF-8
            y=i.split('(')[0]#　只取景點括號前面的字
            if y in exdic:#　判別是否有在例外狀況裡
                z=exdic[y]#　有的話接住value
            else:
                z=y#　沒的話還是原本的y
            if not i in Dica:#　判別每個景點在大字典是否有存在
                Dica[i] = {}#　若沒存在則新增一個
            for g in jie:#　讀取撈出來的關鍵字, Jie內容(g)的編碼是UTF-8
                cnt+=1
                try:
                    Dica[i][g]= model.similarity(z.decode('utf-8'),g.decode('utf-8'))
                except:
                    Dica[i][g] = 0

    ###　　中途檢查　　###
    #print cnt#　總關鍵字量
    #return Dica#　傳回初步歸類的字典
    ####################
    
    Dicafinal={}#　篩選：0<權重<0.99的結果
    for a in Dica:
        Dicafinal[a]={}#　複製景點
        for b in Dica[a]:
            if Dica[a][b]>0 and Dica[a][b]<0.99:#　0<權重<0.99的留下
                Dicafinal[a][b] = Dica[a][b]#　內容轉移
                
    ###　　中途檢查　　###
    #return Dicafinal#　傳回篩選過的字典
    ####################

    Dic30={}#　篩選：前30名的結果
    for a in Dicafinal:
        Dic30[a]={}#　複製景點
        words = sorted(Dicafinal[a].iteritems(),key=operator.itemgetter(1),reverse=True)
        for b in words[0:30]:#　取前三十名
            Dic30[a][b[0]]=b[1]

    return Dic30
    
###########################################
###########################################

def jiebaNoEng(attraction,rankNum):
    import re
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    words=jieba.cut(f2)  #斷詞
    
    li = []
    for u in words:
        if not re.search('(\w)(\w)',u):
            li.append(u)
    
    n1 = (' '.join(li)).encode('utf-8')
    f.close()
    x = jieba.analyse.extract_tags(n1, topK=rankNum, withWeight=True)    
    li = []
    for i in x:       
        #print i[0].encode('utf-8')#,i[1]
        li.append(i[0].encode('utf-8'))
    return li
#####################################################################


#Step6　把萃取出來的關鍵字作概念分類


#####################################################################
def W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model):
    
    import gensim
    #1__WIKI
    if Model == 'Wiki':
        model = gensim.models.Word2Vec.load("W2V_Wiki_Trastd/wiki.zh.sim.seg.trastd.model")
    #2__搜狐新闻数据
    elif Model == 'SogouCS':
        model = gensim.models.Word2Vec.load('W2V_SogouCS/SogouCS_Combine.model')
    #3__全网新闻数据
    elif Model == 'SogouCA':
        model = gensim.models.Word2Vec.load('W2V_SogouCA/SogouCA_Combine.model')
       
    import operator
    cnt = []#　　計算餵進來的總詞量
    Expt = []#　　存W2V無法辨識的詞語
    Remain = []#　　存沒有通過門檻的詞語
    ClassingResult = {}

    for z in Concept:#　　把概念標頭寫進字典
        ClassingResult[z] = {}#　　每個字典的內容都是一個字典

    #for q in open(Corpus,'r'):#　　把萃取關鍵字的結果讀出來        
    #    q = q.strip().decode('utf-8')#　　去空白,解碼
    for q in Corpus:
        q = q.decode('utf-8')
        cnt.append(q)#　　把讀取內容寫進一陣列(計算樣本的資料量用)
        try:
            TmpDic = {}#　　暫時性的字典(比對詞語與各概念的相似度)
            SameConcept = ''#　　用來存放概念,當q直接是某概念時使用
            TmpWord = 'None'#　　用來比對詞語是不是與概念完全吻合,若是,None會被修改掉;反之則維持None(初值)

            for a in Concept:#　　讀取各概念標頭
                TmpList = []#　　暫時性的陣列(存放'詞語'vs'概念內各詞語'的相似度)     
                for b in Concept[a]:#　　讀取個概念內的詞語

                    #####完全ㄧ樣#####
                    if q.encode('utf-8') == b:#　　如果等於
                        #print '---目標詞「',q,'」就在【',a,'】的概念裡'
                        TmpWord = b#　　把詞語指定給TmpWord
                        break#　　跳出迴圈(已經知道跟某詞語完全一樣,就不用再比其他的詞語了)
                    #####

                    if model.similarity(q,b.decode('utf-8')) > SimLevel:#　　如果相似度達門檻
                        TmpList.append(model.similarity(q,b.decode('utf-8')))#　　就把相似度寫進TmpList裡面

                #####完全一樣#####        
                if TmpWord != 'None':#　　不是None(初值)->被修改過->要完全相等才會出現此情況
                    SameConcept = a#　　把現在比對到的概念( EX:輕鬆 )記起來
                    break#　　跳出迴圈(已經知道是屬於哪個概念,就不用再比對其他概念了)
                #####

                if float(len(TmpList))/float(len(Concept[a])) > SimPercent:#　　如果達到相似成數的門檻
                    #print '  -「',q,'」已過相似_百分比_門檻 :',SimPercent
                    TmpAvg = 0.0#　　存放平均相似度
                    for c in TmpList:#　　把每個相似度讀出來
                        TmpAvg += c#　　加起來
                    TmpDic[a] = float(TmpAvg)/float(len(TmpList))#　　把'總數'除以'陣列長度',得到平均相似度,
                                                                    #再寫進字典裡,當q與多個概念都有相似度食
                                                                    #可以比較與哪個概念的相似度比較高

            #####完全一樣##### 
            if TmpWord != 'None': 
                ClassingResult[SameConcept][(q.encode('utf-8'))] = 1#　　把q寫進總字典的概念裡(Key:q Value:1)
                continue
            #####

            if len(TmpDic) > 0:#　　如果暫存字典有任何一概念(表示q'至少'與一個概念相似)
                #print '> >小結 :',q,'成功歸類！',
                for d in TmpDic:#　　遍歷TmpDic
                    if d == sorted(TmpDic.iteritems(),key=operator.itemgetter(1),reverse=True)[0][0]:
                        #如果d = 排序後的第一名 (表示d就是與q最像的概念)
                        if TmpDic[d] > SimLevel:#　　如果"q與概念的相似度"大於相似度門檻
                            #print '類別為 :',d,'\n'
                            ClassingResult[d][q.encode('utf-8')] = TmpDic[d]#　　把q寫進總字典的概念裡(Key:q Value:相似度)
                        else:#　　不太可能發生的情況
                            Remain.append('WHAT!?'+q.encode('utf-8'))                    
            else:#　　q與各概念相似度都沒有達標,無法歸類
                Remain.append(q.encode('utf-8'))

        except KeyError:#　　抓到KeyError( W2V不認識q )的例外處理
            #print '「',q,'」EXCEPT! (KeyError)\n'
            Expt.append(q.encode('utf-8'))#　　把q加進例外的陣列理
            
#####　　把分類結果字典回傳　　#####
    return ClassingResult

##################################################################################################

##################################################################################################

def AttrClassify(DictFinal,PrintReport,PrintDict,ReturnDict,\
                 Concept,SimLevel,SimPercent,Model):
###***************************************************
    Corpus = []
    for i in DictFinal:
        for a in DictFinal[i]:
            Corpus.append(a)
    cnt2 = len(Corpus)
    ClassingResult=W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model)    
###***************************************************


    if PrintDict == 'PrintConcept':
        for ee in ClassingResult:
            print ee,str(ClassingResult[ee]).decode('string_escape')
    
    
    DictResult={}#　把景點關鍵字的結果轉成'概念'.'文字雲'
    for a in DictFinal:#　把{景點:[關鍵字]}的字典讀出來
        DictResult[a] = {'概　念':{},'文字雲':[]}#　指定KEY:景點 Value:概念{} 文字雲

        for b in DictFinal[a]:#　把每個景點的關鍵字讀出來
            z = 'Homeless'#　先假設每個關鍵字都沒有家

            for c in ClassingResult:#　讀取每個概念,內容是{概念:{詞語:比重}}讀出來
                if b in ClassingResult[c]:#　如果b(現在讀到的這個關鍵字)在某概念裡面
                    z = 'Home'#　有家了

                    for e in ClassingResult[c]:#　把概念的詞都讀一讀
                        if b == e:#　如果'概念內的詞'='現在的關鍵字'-->抓到了!
                            if not c in DictResult[a]['概　念']:#　如果景點還沒有此概念
                                DictResult[a]['概　念'][c]={}#　為景點的此概念新增一個字典
                                DictResult[a]['概　念'][c][b] = ClassingResult[c][e]# 並且把字詞寫進此景點的概念裡,KEY:字詞 Value:字詞與概念的相似度    
                            else:#　如果景點已經有此概念了
                                DictResult[a]['概　念'][c][b] = ClassingResult[c][e]#　把字詞寫進此景點的概念裡,KEY:字詞 Value:字詞與概念的相似度
                            break#　因為以經分好家了,寫完之後就跳出迴圈
                    break#　因為已分好家,所以再次跳出迴圈

            if z == 'Homeless':#　如果字詞並沒有屬於任何概念,依然沒有家
                DictResult[a]['文字雲'].append(b)#　就把字詞寫進該景點的'文字雲'裡面
                
                
    #驗證結果
    if PrintReport == 'Y':
        Class = 0
        Cloud = 0
        for g in DictResult:
            for h in DictResult[g]:
                if h == '文字雲':
                    Cloud += len(DictResult[g][h])
                if h == '概　念':
                    for k in DictResult[g][h]:
                        Class += len(DictResult[g][h][k])
                        
        print 'DictReSult內含詞語總量: ',(Class + Cloud)
        if (Class + Cloud) == cnt2:
            print '4. DictReSult寫入(轉換)成功! 並無遺漏!\n'
        else:
            print '4. 錯誤! DictReSult寫入(轉換)過程出現遺漏!\n'


    if PrintDict == 'PrintC':
        for g in DictResult:
            print '【',g,'】',
            for u in DictResult[g]['概　念']:
                print u,
            print '\n'        
            
    if PrintDict == 'PrintD':
        for g in DictResult:
            print '【',g,'】'
            for h in DictResult[g]:
                print '｢',h,'｣'

                if h == '概　念':
                    for k in DictResult[g][h]:
                        print k,len(DictResult[g][h][k]),str(DictResult[g][h][k]).decode('string_escape')
                elif h == '文字雲':
                    print str(DictResult[g][h]).decode('string_escape')
            print ''

    ###決定是否回傳字典
    
    if ReturnDict == 'ReturnC':
        return DictResult
    elif ReturnDict == 'ReturnConcept':
        return ClassingResult

In [2]:
###　　　概念字典
Dic = {
'藝文':['藝術','美術','戲劇','音樂'],\
    
'信仰':['信仰','祭祀','祭拜','膜拜','信眾','信奉'],\
    
'美食':['美食','爽口','美味','好吃'],\
'新奇':['新奇','新穎','獵奇','獨特','驚喜'],\

'輕鬆':['輕鬆','休閒','遊憩','悠閒','休憩','消遣','愜意','放鬆'],\
'遊玩':['遊玩','好玩','遊樂'],\
'快樂':['快樂','愉快','開心','歡樂','幸福','美好'],\
'有趣':['有趣','趣味'],\

'脫俗':['脫俗','幽雅','清幽','優雅','氣質','心曠神怡'],\
'樂活':['環保','綠化','清新'],\

'安靜':['安靜','寧靜','靜謐','平靜','鳥鳴'],\
'乾淨':['乾淨','整潔','潔淨'],\

'懷舊':['懷舊','復古','情調'],\
'古老':['古老','悠久'],\

'髒亂':['骯髒','很髒','髒亂','低劣','灰塵','破舊'],\

'購物':['商圈','百貨','商場'],\
'特產':['特產','紀念品'],\

'健康':['運動','健康'],\

'漂亮':['漂亮','可愛','美麗','優美'],\

'壯觀':['雄偉','壯觀','壯麗']
}

#####################################################################
###　　　例外狀況

exdic={'佳佳唱片行':'佳佳','王師傅餅鋪':'王師傅','草山金工':'金工','維格餅家':'維格','捷運北門站':'北門',\
        '貓空親山步道':'貓空','猴山岳':'貓空','台北書畫院':'書畫院','台北醫學大學':'北醫','高地公園':'高地',\
        '櫻崗溫泉會館':'櫻崗','台北科技大學':'北科','陽明山菁山遊憩區':'菁山','台北永樂市場':'永樂市場',\
        '橫嶺古道':'橫嶺','北投':'北投公園露天溫泉','花博故事館':'花博','外雙溪頂山':'外雙溪','陽明山前山公園':'前山公園',\
        '羅漢窟溫泉':'北投','林泉公園':'林泉','台北世界貿易中心第三館':'世界貿易中心','桃源街牛肉麵':'桃源街',\
        '艋舺電影拍攝場景':'艋舺','天母生活市集':'市集','花博文化館':'花博','胡思二手書店':'胡思',\
        '三墩石茶壺博物館':'三墩','振華公園':'振華','天母公園':'天母','玉成公園':'玉成','聖多福天主堂':'天主堂',\
        '臨濟護國禪寺':'禪寺','林果良品':'林果','毛毛蟲兒童哲學基金會':'毛毛蟲','台鐵夢工場':'夢工場',\
        '內溝溪景觀生態步道':'內溝溪','永春二手市集':'市集','翠峰步道':'翠峰','登峰圳':'登峰','大同親子館':'親子',\
        '平溪步道':'平溪','麗山有機農場':'有機','文化大學後山':'文大','坪頂碼礁步道':'步道','圳堤古道':'古道',\
        '牛奶湖山頭公園':'牛奶湖','發現生活園藝':'園藝','新光三越百貨南西店':'百貨公司','祖師禪林':'祖師',\
        '陽明山櫻花林':'櫻花','永康商圈':'永康','社子島島頭公園':'社子島','北投天主堂':'天主堂','中央藝文公園':'藝文',\
        '文間山':'文間','江記華隆商行':'商行','平安鐘':'平安','翠峰瀑布':'翠峰','台北濱江街飛機巷':'濱江街',\
        '華光社區':'華光','雄獅永康人文空間':'人文','叢林樂園':'叢林','國立台灣藝術教育館':'藝術',\
        '秋惠文庫':'秋惠','張迺妙茶師紀念館':'茶師','仙跡岩':'仙跡','台灣昆蟲館':'昆蟲','大崙頭山':'大崙',\
        '大稻埕戲苑':'戲苑','敘舊布袋戲園':'布袋戲','巧克力奇幻世界':'巧克力','台北師大附中':'師大附中',\
        '北投法藏寺':'法藏','硫磺谷遊憩區':'硫磺','湯屋溫泉':'溫泉',' 大稻埕辜宅':'辜宅','蘇荷兒童美術館':'美術館',\
        '國立陽明大學':'陽明大學','黨史館':'黨史','新坡嶺糶米古道':'新坡嶺','天母跳蚤市場':'跳蚤市場',\
        '翠山公園':'翠山','中國文化大學華岡博物館':'華岡','誠品西門':'誠品','山水香庭園野菜屋':'野菜',\
        '蒙藏文物展示中心':'蒙藏','尼克的香草花園':'花園','陳氏宗祠':'宗祠','建成圓環商場':'商場',\
        '機械恐龍南港車站':'恐龍','台灣總督府交通局鐵道部':'總督府','公館水岸公園':'水岸','原住民文化主題公園':'原住民',\
        '市長官邸':'官邸','小溪頭環狀步道':'環狀','陽明山後山公園':'陽明山','台北信義親子館':'親子',\
        '華納威秀廣場':'華納威秀','五鐵秋葉原':'秋葉原','防災科學教育館':'防災','千禧湯':'千禧','水舍銀光':'水舍',\
        '台北悠活村':'悠活','西門町圓環':'西門町','雙溪水岸腳踏車道':'雙溪','花博養生館':'花博','147漆彈主題樂園':'147',\
        '大崙尾山':'大崙尾','遠企購物中心':'遠企','牯嶺公園':'牯嶺','富民生態公園':'生態',\
        '西門町電影公園':'電影街','龍都冰果專業家':'冰果'}

In [3]:
# Step 1 　從資料庫把東西倒出來
# [一] #
###　　　資料庫連結資訊　　　###
IP   = "10.120.26.4"
User = "iii"
PWd  = "iii"
DB   = "iii"
###　　　###　　　###　　　####
# [二] #
######　　　SQL命令　　　######
sql="SELECT a.A_id, a.A_content, b.Attr_id, c.Attr_name\
     FROM  `iiimap_art_to_att` AS b\
     LEFT JOIN  `iiimap_article` AS a ON a.A_id = b.Article_id\
     LEFT JOIN  `iiimap_attraction` AS c ON b.Attr_id = c.Attr_id\
     WHERE a.attr_in_art_count >=1;"

#############################################################

######　　　SQL命令　　　######(倒遊記)
sqlart = "SELECT a.A_id, a.A_content, b.Attr_id, c.Attr_name\
         FROM  `iiimap_art_to_att` AS b\
         LEFT JOIN  `iiimap_article` AS a ON a.A_id = b.Article_id\
         LEFT JOIN  `iiimap_attraction` AS c ON b.Attr_id = c.Attr_id\
         WHERE a.attr_in_art_count >=1;"

#############################################################

######　　　SQL命令　　　######(倒評論)
sqlcom = "SELECT b.Attr_id, a.Attr_name, b.C_content\
          FROM  `iiimap_attraction` AS a\
          JOIN  `iiimap_comment` AS b ON a.Attr_id = b.Attr_id;"

#############################################################
import time
Date = time.strftime('%Y%m%d',time.localtime(time.time()))

#　訓練W2V model(用遊記+評論)
######　　待訓練檔(原始檔)　　######
InputFile = 'AnCCombine_'+Date+'.txt'

######　　待訓練檔(斷詞後)　　######
OutputFile = 'AnCW2V_'+Date+'.txt'

#############################################################


# Step 2 　淬取關鍵字
# [一] #
######　　決定淬取排名　　######
Rank = 30
W2VModel = OutputFile.split('.txt')[0]+'.model'
# [二] #
######　　　讀取字典　　　######
import os
import jieba
import jieba.analyse
import jieba.posseg as pseg  
jieba.set_dictionary('dict.big.tra.txt')  #預設字典
jieba.load_userdict('dict_twstd_tfidf.txt')  #中文分詞詞庫(TFIDF)
jieba.load_userdict('dict_ntusd_pos.txt') #NTUSD_負向
jieba.load_userdict('dict_ntusd_nag.txt') #NTUSD_正向
jieba.load_userdict('dict_twedu_dict.txt') #教育部《重編國語辭典 修訂本》
jieba.load_userdict('dict_Custom_20160114.txt') # 自定義字典
jieba.analyse.set_stop_words("Stop_Edit_Final_20160131.txt") #停用詞
#############################################################


# Step 3 　更新景點標籤
# [一] #
######　　　輸出相關參數　　　######
PrintReport = 'N'
PrintDict = 'N'
ReturnDict = 'ReturnC'
# [二] #
######　　　W2V相關參數　　　######
Concept = Dic
SimLevel = 0.4
SimPercent = 0.49
Model = 'SogouCS'
#############################################################


# Step 4 　決定重要資訊
######　　　　　放置遊記的資料夾　　　　######
Folder = 'Article_'+Date

######　放置遊記&評論的資料夾(訓練W2V)　######
Folder2 = 'AnCForW2V_'+Date

######　　　存放標籤後結果的JSON檔　　　######
JsonFile = 'ArticleTags-'+Date+'.json'
#############################################################

Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
DEBUG:jieba:Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
DEBUG:jieba:Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
Loading model cost 0.261 seconds.
DEBUG:jieba:Loading model cost 0.261 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [4]:
import json
import time
a1 = time.time()

squary(IP,User,PWd,DB,sql,Folder)
b1 = time.time()
print 'Step1　Dump All Article!...　cost:',b1-a1

art(IP,User,PWd,DB,sqlart,Folder2)
c1 = time.time()
print '\nStep2-1　Dump All Article For W2V...　cost:',c1-b1

com(IP,User,PWd,DB,sqlcom,Folder2)
d1 = time.time()
print '\nStep2-2　Dump All Comment For W2V...　cost:',d1-c1

CombineAnC(Folder2,InputFile)
e1 = time.time()
print '\nStep3　Combine Files For W2V...　cost:',e1-d1

TrainW2V(InputFile,OutputFile)
f1 = time.time()
print '\nStep4　Train W2Vmodel From Art&Com...　cost:',f1-e1

DictFinal = artsco(W2VModel,Folder,Rank,exdic)
g1 = time.time()
print '\nStep5　Extract Keywords Form Article...　cost:',g1-f1

Result = AttrClassify(DictFinal,PrintReport,PrintDict,ReturnDict,\
                      Concept,SimLevel,SimPercent,Model)
h1 = time.time()
print '\nStep6　Classifying KeyWords...　cost:',h1-g1

JResult = json.dumps(Result)#　丟進一JSON格式的變數
with open(JsonFile,'w') as JF:#　寫進Json檔裡
    JF.write(JResult)

print '\nFinish!...　Total Cost:',h1-a1

Articles Be Downloaded: 11885
Step1　Dump All Article!...　cost: 45.5319998264
Total Reading Articles(Not Distinct): 31690

Step2-1　Dump All Article For W2V...　cost: 88.7620000839
Total Reading Comments: 52660

Step2-2　Dump All Comment For W2V...　cost: 105.872999907

Step3　Combine Files For W2V...　cost: 2.36900019646
Read Lines: 84350
InputFile Len: 85028
OutputFile Len: 84349

Step4　Train W2Vmodel From Art&Com...　cost: 364.825999975

Step5　Extract Keywords Form Article...　cost: 133.544999838

Step6　Classifying KeyWords...　cost: 17.6510000229

Finish!...　Total Cost: 758.557999849


　
　
　
# TESTING ========================

測試方法：檢視標籤後的景點結果

WatchTagResultForDic(aa)

    aa：[字典]。[傳入使用過AttrClassify()，使用Result接住ReturnC的結果字典]。
        使用後會印出相關的數據

In [5]:
def WatchTagResultForDic(aa):#　字典版
    ##########################################################
    DicCT = {}#　存放概念的字典(KEY:景點　VALUE:標籤(陣列))
    ConDic = {}#　計算標籤量的字典(KEY:標籤　VALUE:數量)
    AttrNoTags = {}#　存放沒標籤的景點
    for i in aa:#　讀取景點
        for u in aa[i]:#讀取景點的內容
            
#           ###################################################
            if u == '概　念':#　注意！此處為CombineTag的方法改寫！
                            #差別之處是直接拿字典(不是讀取JSON)問
                            #所以此處是用'概　念'(而非u'概　念')
#           ###################################################

                DicCT[i] = []
                if len(aa[i][u])==0:#　如果沒有概念的
                    #print i,aa[i][u]
                    AttrNoTags[i] = aa[i][u]#　加進AttrNoTags裡
                else:#　有概念的
                    for z in aa[i][u]:#　　把概念讀出來
                        DicCT[i].append(z)#　加進字典的VALUE裡
                        if not z in ConDic:#　如果這個概念第一次被讀到
                            ConDic[z] = 1#　加進字典裡
                        else:
                            ConDic[z]+= 1#　字典的計數加一

    print '景點數量:',len(aa)
    print '標籤種類:',len(ConDic)

    c = 0
    ###　　　標籤數　　　###
    for i in ConDic:
        print i,ConDic[i]
        c+=ConDic[i]

    print   '標籤總數:',c
    print   '有　標籤的景點數:',len(aa)-len(AttrNoTags)
    print   '總景點平均標籤數:',float(c)/float(len(aa))#　標籤數/全部景點
    print   '淨景點平均標籤數:',float(c)/float(len(aa)-len(AttrNoTags))#　標籤數/有標籤的景點

    print   '沒有標籤的景點數:',len(AttrNoTags),'\n',\
            '成功標籤比率:',1.0-(float(len(AttrNoTags))/float(len(aa)))

    print '========檢視沒有標籤的景點========'
    for i in AttrNoTags:
        print i

    ###　　把所有景點的標籤寫進一陣列裡（用來做關聯規則）　　###
    F = []
    for i in DicCT:
        F.append(DicCT[i])

In [6]:
###　測試A：針對DictFinal( KEY:景點  VALUE:關鍵字 )
W2VModel ='AnCW2V0131.model'
Folder='Article_20160131'
Rank = 30

DictFinal = artsco(W2VModel,Folder,Rank,exdic)

In [6]:
Corpus=[]
for i in DictFinal:
###　計算：關鍵字總量
    for a in DictFinal[i]:
        Corpus.append(a)

###　檢驗：沒有關鍵字的
    if len(DictFinal[i])==0:
        print i,len(DictFinal[i])

###　檢驗：看單一景點的內容
    #if i == '二二八和平公園':
    #    print i,DictFinal[i]


print len(DictFinal)#　總遊記景點量
print len(Corpus)#　總關鍵字量


林華泰茶行 0
閻錫山墓 0
台北魚市 0
李制餅家 0
福州山公園登山步道 0
松壽廣場 0
郊山友台 0
台灣基督長老教會 0
風尾步道 0
北投親子館 0
稻香公園 0
後火車站懷舊廣場 0
Pentel飛龍文具股份有限公司 0
統領百貨公司 0
西昌街攤販集中場 0
士林公民會館 0
圳仔頭溪自然園區 0
音響機器人 0
國立台北科技大學 0
內雙溪自然中心 0
台北捷運北投站旅遊服務中心 0
康樂街步道 0
康樂街125巷松月櫻 0
馬偕護校 0
黑丸嫩仙草 0
全祥茶莊 0
南天書局 0
錦安市場 0
前港公園 0
內湖運動公園 0
北投公園露天溫泉 0
北投兒童樂園 0
民有3號公園 0
名山公園 0
台北市青少年發展處 0
誠品生活(松煙店) 0
台北捷運劍潭站旅遊服務中心 0
明星咖啡館 0
太原五百完人紀念建築群 0
水源劇場 0
仁慈公園 0
雙溪生活水岸 0
真善美劇院 0
東門福德宮 0
第四屆部落客百傑 0
青青食尚花園會館 0
台北監獄圍牆遺蹟 0
動物溝通密碼特展 0
總統副總統文物館 0
李春生紀念教堂 0
平菁步道 0
南港東新綠地小公園 0
唐山書店 0
市政府耶誕城 0
mymillyzakka米力生活雜貨鋪 0
相信音樂 0
松壽廣場公園 0
內溝溪生態展示館 0
遠東百貨公司 0
南港瓶蓋工廠 0
麗禧酒店 0
台灣神學院 0
台北音樂家書房 0
台大醫學人文博物館 0
春天廚房 0
台一牛奶大王 0
瑩瑋藝術翡翠文化博物館 0
至誠公園 0
台北市花卉批發市場 0
台北北門郵局 0
大稻埕圓環防空蓄水池 0
陽明山美軍宿舍 0
苗榜餐廳 0
天母運動公園 0
朗讀違章 0
台北市青田街 0
台北布事館 0
台灣好店 0
嘉禾新村 0
天母三玉宮 0
台泥大樓 0
景泉浴室 0
豹山溪自然步道 0
老攤鐵板燒 0
丹鳳公園 0
台大醫院西址院區 0
德德小品 0
主婦之家 0
陳悅記祖宅(老師府) 0
蔣渭水紀念公園 0
桂花吊橋 0
永靜公園 0
王貫英紀念圖書館 0
國立台北商業技術學院 0
台北傳奇 0
台灣創價學會至善藝文中心 0
磯永吉小屋 0
MAJI廣場 0
龍泉溫泉 0
天恩宮 0
內湖草莓園 0
復興公園。泡腳池園區 0
台大昆蟲館 0
福德坑環保復育園區 0
內湖休閒農場 0
平安公園 0
台北市立社會

In [20]:
###　測試B：針對Result( KEY:景點  VALUE:概念、文字雲 )
PrintReport = 'N'
PrintDict = 'N'
ReturnDict = 'ReturnC'
Concept = Dic
SimLevel = 0.4
SimPercent = 0.49
Model = 'SogouCS'

Result = AttrClassify(DictFinal,PrintReport,PrintDict,ReturnDict,\
                      Concept,SimLevel,SimPercent,Model)

In [7]:
for i in Result:
    if i =='二二八和平公園':
        print i
        for a in Result[i]:
            print '【',a,'】'
            if a == '概　念':
                if len(Result[i][a])==0:
                    print 'None'
                else:
                    for b in Result[i][a]:
                        print b
            else:
                print ' '.join(Result[i][a])

二二八和平公園
【 概　念 】
None
【 文字雲 】
二二八 襄陽 帥客 愛河 凱道 夜逛 國立博物館 創館 館址 前銅 和平紀念公園 易名 總機 車遊 紀念館 五番 國家圖書館 抽福袋 懷寧 二二八紀念碑 師大附中 對館 即到 植物園 中山堂 衡陽 牛處 忠烈祠 筠園 日貓空


In [8]:
WatchTagResultForDic(Result)

景點數量: 654
標籤種類: 19
遊玩 36
古老 95
購物 27
乾淨 5
壯觀 23
漂亮 36
信仰 16
脫俗 22
樂活 4
安靜 32
快樂 18
美食 95
新奇 5
健康 19
特產 71
懷舊 15
有趣 34
藝文 109
輕鬆 30
標籤總數: 692
有　標籤的景點數: 355
總景點平均標籤數: 1.05810397554
淨景點平均標籤數: 1.94929577465
沒有標籤的景點數: 299 
成功標籤比率: 0.542813455657
========檢視沒有標籤的景點========
京站時尚廣場
芝山公園
雙溪公園
向日葵公園
雙溪生活水岸
芝山岩惠濟宮
真善美劇院
台北兒童新樂園
天母三玉宮
北投市場
林華泰茶行
東門福德宮
梅庭
林語堂故居
告白巷
幾米月亮公車
東方寺
翠山公園
白石湖觀光農園
七星山
碧山巖
康樂街125巷松月櫻
絹絲瀑布
松山彩虹橋
閻錫山墓
新龍公園
西門紅樓
台北監獄圍牆遺蹟
士林官邸公園
士林夜市
眾藝埕
富陽自然生態公園
台北市青少年發展處
延平河濱公園
台北魚市
翠峰瀑布
金龍寺
台泥大樓
北投櫻花道
吳氏宗祠
景泉浴室
動物溝通密碼特展
李制餅家
台北濱江街飛機巷
玉成公園
福州山公園登山步道
台灣菸酒公賣局
松壽廣場
台北101
豹山溪自然步道
老攤鐵板燒
郊山友台
布政使司文物館
夢幻湖
士東市場
台大醫院西址院區
前山公園
聖家堂
陳氏宗祠
德德小品
主婦之家
急公好義坊
機械恐龍南港車站
衛視中文台
陳悅記祖宅(老師府)
忠烈祠
擎天崗
歪腰郵筒
台灣基督長老教會
風尾步道
北投親子館
北投公園
天母士東國小
猴山岳
台大醫學人文博物館
中央氣象局
稻香公園
福德坑環保復育園區
台北轉運站
147漆彈主題樂園
頂山石梯嶺步道
東區地下街
少帥禪園
青年公園
蔣渭水紀念公園
毛毛蟲兒童哲學基金會
後火車站懷舊廣場
Pentel飛龍文具股份有限公司
台北靈糧堂
神農市場
統領百貨公司
新北投捷運站
北投國中
永靜公園
台北故事館
王貫英紀念圖書館
華西街夜市
真相館
軍艦岩親山步道
象山
李春生紀念教堂
台北賓館
平菁步道
松山機場
天母古道親山步道
士林公民會館
台北世界貿易中心南港展覽館
李石樵美術館
胡適公園
南港東新綠地小公園
唐山書店
圳仔頭溪自然園區
于右任故居
市政